# KNN

K-En Yakın Komşu (KNN), bir makine öğrenimi algoritmasıdır. Bir örneğin sınıfını veya değerini belirlemek için en yakın k komşusunun etrafındaki verilere dayanır. Uzaklık ölçüsü kullanarak, belirli bir örneğin k en yakın komşusunu seçer. Sınıflandırma için en yaygın etiketi, regresyon için ise ortalama değeri kullanarak tahmin yapar. Basit, eğitim süresi olmayan bir algoritmadır ancak büyük veri setlerinde performans sorunları yaşayabilir ve aykırı değerlere hassas olabilir.

**1. Exploratory Data Analysis - Keşifsel Veri Analizi**

**2. Data Preprocessing & Feature Engineering - Veri Ön İşleme ve Özellik Mühendisliği**

**3. Modeling & Prediction - Modelleme ve Tahmin**

**4. Model Evaluation - Model Değerlendirme**

**5. Hyperparameter Optimization - Hiperparametre Optimizasyonu**

**6. Final Model - Son Model**

In [1]:
# Pandas kütüphanesini ve gerekli sklearn modüllerini içe aktar
import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
# DataFrame içindeki tüm sütunları ekrana yazdırma seçeneğini etkinleştir
pd.set_option('display.max_columns', None)

# Ekrana yazdırılan sütunların genişliğini artırma
pd.set_option('display.width', 500)


### Exploratory Data Analysis - Keşifsel Veri Analizi

In [3]:
# "diabetes.csv" dosyasını kullanarak bir DataFrame oluştur
df = pd.read_csv("datasets/diabetes.csv")

# DataFrame'in ilk 5 satırını ekrana yazdır
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# DataFrame'in satır ve sütun sayısını ekrana yazdır
df.shape


(768, 9)

In [5]:
# DataFrame'in temel istatistiksel özetini ekrana yazdır
# .T Transpoz işlemidir. geniş veri setlerinde daha iyi okunabilir bir çıktı elde edilir
df.describe().T


,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [6]:
# "Outcome" sütunundaki benzersiz değerlerin sayısını ekrana yazdır
df["Outcome"].value_counts()


Outcome
0    500
1    268
Name: count, dtype: int64

### 2. Data Preprocessing (Veri Ön İşleme) & Feature Engineering (Özellik Mühendisliği)

KNN aykırı değerlere karşı duyarlıdır. Bu yüzden verileri standartlaştırmak gerekir.

In [7]:
# "Outcome" sütununu hedef değişken olarak seç
y = df["Outcome"]

# "Outcome" sütununu hariç tutarak bağımsız değişkenleri seç
X = df.drop(["Outcome"], axis=1)

# Bağımsız değişkenleri standartlaştır
X_scaled = StandardScaler().fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

# Standartlaştırılmış verilerin temel istatistikleri ekrana yazdırılır.
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,-6.476301e-17,1.000652,-1.141852,-0.844885,-0.250952,0.639947,3.906578
Glucose,768.0,-9.251859e-18,1.000652,-3.783654,-0.685236,-0.121888,0.605771,2.444478
BloodPressure,768.0,1.503427e-17,1.000652,-3.572597,-0.367337,0.149641,0.563223,2.734528
SkinThickness,768.0,1.006140e-16,1.000652,-1.288212,-1.288212,0.154533,0.719086,4.921866
Insulin,768.0,-3.006854e-17,1.000652,-0.692891,-0.692891,-0.428062,0.412008,6.652839
BMI,768.0,2.590520e-16,1.000652,-4.060474,-0.595578,0.000942,0.584771,4.455807
DiabetesPedigreeFunction,768.0,2.451743e-16,1.000652,-1.189553,-0.688969,-0.300128,0.466227,5.883565
Age,768.0,1.931325e-16,1.000652,-1.041549,-0.786286,-0.360847,0.660206,4.063716


### 3. Modeling & Prediction - Modelleme ve Tahmin

In [8]:
# K-En Yakın Komşular Sınıflandırıcı modelini oluştur ve eğit
knn_model = KNeighborsClassifier().fit(X, y)


In [9]:
# Rastgele bir kullanıcı örneği seç ve KNN modeli ile tahmin yap
# random_state=45 yaparsanız benim ile aynı sonucu elde edersiniz.
random_user = X.sample(1, random_state=45)

# KNN modeli ile tahmin yap
knn_model.predict(random_user)


array([1], dtype=int64)

### 4. Model Evaluation - Model Değerlendirme



In [10]:
# Confusion matrix oluşturmak için y_pred'i elde et
y_pred = knn_model.predict(X)


KNN modelinin bağımsız değişkenler üzerinde sınıf olasılıklarını tahmin etmesini sağlar. **predict_proba(X)** fonksiyonu, her bir sınıf için olasılıkları içeren bir matris döndürür. **[ :, 1]** ifadesi, sadece pozitif sınıfın olasılıklarını içeren sütunu seçer. Bu olasılıklar genellikle AUC (Alan Altındaki Eğri) hesaplamak gibi performans metriklerinde kullanılır.

In [11]:
# AUC hesaplamak için y_prob'u elde et
y_prob = knn_model.predict_proba(X)[:, 1]


In [12]:
# Sınıflandırma raporunu ekrana yazdır
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



In [13]:
# ROC AUC değerini hesapla
roc_auc_score(y, y_prob)


0.9017686567164179

Bizim için önemli olan parametreler;

**Accuracy(Kesinlik)** ==> 0.83 Toplam başarılı tahmin oranı

**Precision (Hassasiyet)** ==> 0.79 Tahmini 1 olanların gerçekten 1 olma oranı

**Recall (Duyarlılık)** ==> 0.70 Gerçekte 1 olanların 1 olarak tahmin edilme oranı

**F1-Score** ==> 0.74 Hassasiyet ve duyarlılık ortalaması

**ROC AUC** ==> 0.90 Bir sınıflandırma modelinin binary (iki sınıflı) sınıflandırma performansını ölçen bir metriktir.

Modelin iyi eğitildiğinden emin olmak için 5 katlı çapraz doğrulama uygulayıp sonuçların ortalamasına göz atalım.


In [14]:
# KNN modeli üzerinde çapraz doğrulama sonuçlarını al
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

In [15]:
# Çapraz doğrulama sonuçlarından ortalama accuracy değerini al
cv_results['test_accuracy'].mean()

0.733112638994992

In [16]:
# Çapraz doğrulama sonuçlarından ortalama F1 score değerini al
cv_results['test_f1'].mean()

0.5905780011534191

In [17]:
# Çapraz doğrulama sonuçlarından ortalama ROC AUC değerini al
cv_results['test_roc_auc'].mean()

0.7805279524807827

Baktığımızda bulduğumuz değerler 5 katlı çapraz doğrulamadan elde edilen ortalamalardan yüksek 

Peki bu değerleri arttırmak için ne yapabiliriz;
--
    1- Örnek boyutunu arttırmak
    2- Veri ön işleme
    3- Özellik mühendisliği
    4- İlgili algoritma için optimizasyonlar
    
KNN algoritması için optimizasyon yapacak olursak;


In [18]:
# KNN modelinin parametrelerini ekrana yazdır
knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

Biz herhangi bir iyileştirme yapmadığımızda modelin parametreleri yukarıda görüldüğü gibidir. Şimdi modeli daha iyi hale getirmek için hiperparametreleri belirleyelim.

### 5. Hyperparameter Optimization -  Hiperparametre Optimizasyonu


In [19]:
# Yeni bir KNN modeli oluştur
knn_model = KNeighborsClassifier()

# KNN modeli için değerlendirilecek hiperparametre aralığını belirle
knn_params = {"n_neighbors": range(2, 50)}

# GridSearchCV kullanarak en iyi hiperparametreleri bul
knn_gs_best = GridSearchCV(knn_model,
                           knn_params,
                           cv=5,
                           n_jobs=-1,
                           verbose=1).fit(X, y)

# En iyi hiperparametreleri ekrana yazdır
knn_gs_best.best_params_


Fitting 5 folds for each of 48 candidates, totalling 240 fits


{'n_neighbors': 17}

Hiperparametre aralığını 2-50 olarak belirledik ve 5 katlı CV uyguladık. Yani 48 adayın her biri için 5 kat yerleştirme yapılarak toplamda 240 yerleştirme uygulandı. Bu işlem sonucunda en iyi **n_neighbors** parametresini **17** olarak belirledik. 

Daha önceki modelimizde bu değer 5'di. Şimdi bu parametre değerlerini modele tanımlayarak modelimizi yeniden oluşturulalım


### 6. Final Model

In [20]:
# En iyi hiperparametreleri kullanarak final KNN modelini oluştur ve eğit;
# **şeklinde kullanılma sebebi parametre sayısı birden fazla olabilir. Hepsini tek tek 
# girmemek için bu şekilde kullanılır.
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X, y)

# Çapraz doğrulama sonuçlarını al
cv_results = cross_validate(knn_final,
                            X,
                            y,
                            cv=5,
                            scoring=["accuracy", "f1", "roc_auc"])

#scoring parametresi, hangi performans metriklerini değerlendireceğimizi belirler.

In [21]:
# Çapraz doğrulama sonuçlarından ortalama accuracy değerini al
cv_results['test_accuracy'].mean()


0.7669892199303965

In [22]:
cv_results['test_f1'].mean()

0.6170909049720137

In [23]:
cv_results['test_roc_auc'].mean()

0.8127938504542278

Hiperparametreleri optimize ettikden sonra 

**Accuracy(Kesinlik)** ==> Eski Değer = **0.73** Yeni Değer = **0.76**

**F1-Score** ==> Eski Değer = **0.59**  Yeni Değer = **0.61**

**ROC AUC** ==> Eski Değer = **0.78** Yeni Değer = **0.81**

Görüldüğü üzere KNN algoritması için hiperparametreleri optimize ettiğimizde başarı oranımız arttı.

In [24]:
# Rastgele bir kullanıcı örneği seç ve KNN modeli üzerinde tahmin yap
# random_state = 42 yaparsanız aynı sonucu elde edersiniz.
random_user = X.sample(1, random_state=42)
knn_final.predict(random_user)


array([0], dtype=int64)